In [3]:
!pip install xgboost
!pip install mlxtend
!pip install jieba
!pip install textblob

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from datetime import date, timedelta
import calendar
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
import jieba
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler


## 超參數

In [5]:
config={
    'Train_Set':'intern_homework_train_dataset.csv',
    'Public_Test_Set':'intern_homework_public_test_dataset.csv',
    'Private_Test_Set':'intern_homework_private_test_dataset.csv',
}

param = {
    'max_depth': 9,
    'eta': 0.05,
    'objective': 'reg:squarederror',
    'eval_metric': 'mape',
}
num_round = 300

# Function

In [6]:
def join_df(train, test,Private_set):
  df = pd.concat([train, test,Private_set], axis=0).reset_index(drop = True)
  features = [c for c in df.columns if c not in [TARGET_COL]]
  df[num_cols + ['like_count_24h']] = df[num_cols + ['like_count_24h']].apply(lambda x: np.log1p(x))
  return df, features
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True):
    seg_list =jieba.cut(text)
    text = " ".join(seg_list)
    return text

def calculate_correlation(row):
    like_counts = [row[f'like_count_{i}h'] for i in range(1, 7)]
    comment_counts = [row[f'comment_count_{i}h'] for i in range(1, 7)]
    return np.corrcoef(like_counts, comment_counts)[0, 1]
def cal_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Data


In [7]:
train=pd.read_csv(config['Train_Set'])
train.head()

,title,created_at,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats,like_count_24h
0,我的排骨湯,2022-10-05 14:20:21 UTC,12,15,15,15,16,18,10,10,10,10,10,10,598518,428921,0.7,26
1,#請益 婚禮穿搭,2022-10-05 14:28:13 UTC,0,0,3,4,4,4,2,5,8,9,9,9,399302,650840,63.9,11
2,無謂的啦啦隊,2022-10-06 07:18:22 UTC,3,7,8,11,12,14,1,1,2,3,3,3,650776,717288,19.2,19
3,文學理論 課本,2022-09-20 11:39:14 UTC,2,7,11,24,26,26,2,2,8,32,38,63,471023,173889,7.9,29
4,一般課程,2022-09-05 10:18:24 UTC,3,7,7,10,10,11,15,26,35,38,48,49,230184,594332,36.2,16


In [8]:
test=pd.read_csv(config['Public_Test_Set'])
test.head()

,title,created_at,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats,like_count_24h
0,分開是為了更好的相遇,2022-12-10 12:14:06 UTC,2,7,7,12,13,14,0,1,1,1,2,2,399368,298421,48.6,16
1,好想交男朋友,2022-11-10 00:54:29 UTC,1,1,1,1,3,4,18,18,23,26,27,28,817817,623425,389.1,8
2,認識四年的朋友正在追我,2022-11-02 07:46:45 UTC,2,2,3,3,3,4,4,4,8,9,11,11,817817,749002,389.1,8
3,購物節電子發票（紙本）驗證失敗,2022-11-25 05:26:40 UTC,0,0,1,2,3,3,0,5,8,14,16,16,828054,857568,32.7,6
4,#閒聊 IVE和張員瑛的病毒式行銷⋯？,2022-11-06 04:38:39 UTC,12,21,39,54,77,107,6,16,21,21,23,28,626235,500634,44.6,211


In [9]:
Private_Test_set=pd.read_csv(config['Private_Test_Set'])
Private_Test_set.head()

,title,created_at,like_count_1h,like_count_2h,like_count_3h,like_count_4h,like_count_5h,like_count_6h,comment_count_1h,comment_count_2h,comment_count_3h,comment_count_4h,comment_count_5h,comment_count_6h,forum_id,author_id,forum_stats
0,#心情 頂樓風好大,2023-02-23 00:18:49 UTC,6,9,10,12,12,16,4,8,9,11,12,12,739682,145559,11.0
1,語言學習夥伴,2023-01-17 17:09:01 UTC,0,1,1,1,1,1,18,32,39,39,39,39,136811,449821,27.5
2,#空間氛圍 開學調時差🕰️13個私藏的睡眠撇步,2023-02-11 11:09:32 UTC,1,2,2,2,2,2,0,1,1,1,1,1,894031,314748,16.8
3,【請益】半導體業務職缺,2023-01-03 15:16:51 UTC,0,0,1,3,3,3,2,5,6,6,6,6,852766,198588,27.8
4,#心情抒發 上班第一周，感覺同事都很認真，完全不敢請假,2023-01-13 23:45:14 UTC,0,1,2,3,3,5,0,2,5,8,10,17,531929,461978,16.8


# Preprocess

In [10]:
TARGET_COL ='like_count_24h'
num_cols = ['like_count_1h','like_count_2h','like_count_3h','like_count_4h','like_count_5h','like_count_6h','comment_count_1h','comment_count_2h','comment_count_3h','comment_count_4h','comment_count_5h','comment_count_6h','forum_stats']
cat_cols = ['forum_id','author_id']
text_cols = ['title']
date_cols = ['created_at','date_only','time_only']

In [11]:
df, features = join_df(train,test,Private_Test_set)
train['set'] = 'train'
test['set']='test'
Private_Test_set['set'] = 'Private_Test_set'
df = pd.concat([train, test,Private_Test_set])

## 類別

In [12]:
df = pd.get_dummies(df, columns=cat_cols)
df = df.fillna(0)
df[num_cols + ['like_count_24h']] = df[num_cols + ['like_count_24h']].apply(lambda x: np.log1p(x))

# Feature engineering

## 處理時間

In [13]:
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_localize(None)
df['created_at'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d %H:%M:%S')
df['created_at_since_start'] = (df['created_at'] - df['created_at'].min()).dt.days
df['created_at_of_week'] = df['created_at'].dt.dayofweek
df['created_at_year'] = df['created_at'].dt.year
df['created_at_year_month'] = df['created_at'].dt.month
df['created_at_day'] = pd.to_datetime(df['created_at']).dt.day
df['created_at_hour'] = pd.to_datetime(df['created_at']).dt.hour

## Title

In [14]:
df['title_len'] = df['title'].apply(lambda x: len(x))
df["clean_title"] = df["title"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, ))
df['clean_title_word_count'] = df["clean_title"].apply(lambda x: len(str(x).split(" ")))
df['clean_title_char_count'] = df["clean_title"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
df['clean_title_avg_word_length'] = df['clean_title_char_count'] / df['clean_title_word_count']
df["clean_title_sentiment"] = df['clean_title'].apply(lambda x: TextBlob(x).sentiment.polarity)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.354 seconds.
Prefix dict has been built successfully.


In [15]:
vectorizer = TfidfVectorizer(tokenizer=utils_preprocess_text, min_df=5, max_df=0.8, ngram_range=(1, 2), max_features=500)
title_tfidf_matrix = vectorizer.fit_transform(df['title'])
title_tfidf_df = pd.DataFrame(title_tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), title_tfidf_df], axis=1)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## like && comment

In [16]:
for i in range(1, 6):
    df[f'like_count_diff_{i}_{i+1}h'] = df[f'like_count_{i+1}h'] - df[f'like_count_{i}h']
    df[f'comment_count_diff_{i}_{i+1}h'] = df[f'comment_count_{i+1}h'] - df[f'comment_count_{i}h']
for i in range(1, 7):
    df[f'like_count_ratio_{i}h'] = df[f'like_count_{i}h'] / (df[f'like_count_{i}h'].sum() + 1e-8)
    df[f'comment_count_ratio_{i}h'] = df[f'comment_count_{i}h'] / (df[f'comment_count_{i}h'].sum() + 1e-8)
for i in range(1, 7):
    df[f'like_comment_ratio_{i}h'] = df[f'like_count_{i}h'] / (df[f'comment_count_{i}h'] + 1e-8)
    df[f'like_comment_diff_{i}h'] = df[f'like_count_{i}h'] - df[f'comment_count_{i}h']
df['like_comment_corr'] = df.apply(calculate_correlation, axis=1)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


## split

In [17]:
train = df[df['set']=='train']
test = df[df['set']=='test']
train = train.drop('set', 1)
test = test.drop('set', 1)
Private_Test_set = df[df['set']=='Private_Test_set']
Private_Test_set = Private_Test_set.drop('set', 1)
features = [c for c in train.columns if c not in [TARGET_COL]]
cat_num_cols = [c for c in features if c not in ['title','created_at', 'clean_title']]
X = train[features]
y = train[TARGET_COL]
Private_Test_set=Private_Test_set.drop(TARGET_COL, 1)
X_train, X_val, y_train, y_val = train_test_split(train[cat_num_cols],y, test_size=0.2, random_state = 23)

C:\Users\User\AppData\Local\Temp\ipykernel_4560\3292943118.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop('set', 1)
C:\Users\User\AppData\Local\Temp\ipykernel_4560\3292943118.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop('set', 1)
C:\Users\User\AppData\Local\Temp\ipykernel_4560\3292943118.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Private_Test_set = Private_Test_set.drop('set', 1)
C:\Users\User\AppData\Local\Temp\ipykernel_4560\3292943118.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Private_Test_set=Private_Test_set.drop(TARGET_COL, 1)


## outbound

In [18]:
y_train_mean = np.mean(y_train)
y_train_std = np.std(y_train)
sample_weights = np.ones_like(y_train, dtype=np.float32)
#std2_outliers = (np.abs(y_train - y_train_mean) >= 2 * y_train_std) & (np.abs(y_train - y_train_mean) < 3 * y_train_std)
std2_outliers = (np.abs(y_train - y_train_mean) >= 2 * y_train_std)
#std3_outliers = np.abs(y_train - y_train_mean) >= 3 * y_train_std
sample_weights[std2_outliers] = 0.7
#sample_weights[std3_outliers] = 0.1

# XGBoost

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train, weight=sample_weights)
dtest = xgb.DMatrix(X_val, label=y_val)
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, 'train'), (dtest, 'test')], early_stopping_rounds=5)

[0]	train-mape:0.76584	test-mape:0.76745


KeyboardInterrupt: 

In [ ]:
tes_matrix=xgb.DMatrix(test[cat_num_cols])
test_pre=bst.predict(tes_matrix)
test_pre = np.round(np.expm1(test_pre)).astype(int).ravel()
test_true=np.round(np.expm1(test[TARGET_COL])).astype(int)
print(cal_mape(test_true,test_pre))

# Private_Test_set save

In [ ]:
pri_tes_matrix=xgb.DMatrix(Private_Test_set[cat_num_cols])
pri_xgboost=bst.predict(pri_tes_matrix)
pri_xgboost = np.round(np.expm1(pri_xgboost)).astype(int).ravel()
Private_Test_pred = pd.DataFrame({'like_count_24h': pri_xgboost})
Private_Test_pred.to_csv('result.csv', index=False)